In [2]:
!pip install tensorflow-gpu
!pip install tensorflow-addons
!pip install ffmpeg

ERROR: Could not find a version that satisfies the requirement tensorflow-gpu (from versions: none)
ERROR: No matching distribution found for tensorflow-gpu


In [35]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [15]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import matplotlib
import ffmpeg
import gzip
import random

from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

class MMNISTDataset(tf.keras.utils.Sequence):

    def __init__(self, data_set):
        super(MMNISTDataset, self).__init__()
        self.data_set = data_set
        self.batch_size = 16
        
        if data_set == "train":
          self.length = int(9008/self.batch_size)
          f = gzip.open('pa2_data/train-images-idx3-ubyte.gz','r')
          image_size = 28
          num_images = 60000
          f.read(16)
          buf = f.read(image_size * image_size * num_images)
          self.mnist_train_im  = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
          self.mnist_train_im  = self.mnist_train_im.reshape(num_images, image_size, image_size)
        elif data_set == "test":
          self.length = int(9008/self.batch_size)
          test_set = np.load('pa2_data/MNIST_test_seq.npy')
          print(test_set.shape)
          test_set = test_set.transpose(1,0,2,3)
          self.mnist_test = test_set[0:9008]
          self.mnist_test = np.expand_dims(self.mnist_test, axis=4)
        elif data_set == "valid":
          self.length = int(992/self.batch_size)
          test_set = np.load('pa2_data/MNIST_test_seq.npy')
          test_set = test_set.transpose(1,0,2,3)
          self.mnist_valid = test_set[9008:10000]
          self.mnist_valid = np.expand_dims(self.mnist_valid, axis=4)

    def __len__(self):
        return self.length

    def gen_random_sequence(self):
        ''' randomly generate a sequence of a digit '''
        size = 64 - 28
        x, y, theta = random.random(), random.random(), random.random() * 2 * np.pi
        velocity_y, velocity_x = np.sin(theta), np.cos(theta)
        seq_x, seq_y = np.zeros(20), np.zeros(20)
        for i in range(20):
            y += 0.1*velocity_y
            x += 0.1*velocity_x
            if x <= 0:
                x = 0
                velocity_x = -velocity_x
            if x >= 1.0:
                x = 1.0
                velocity_x = -velocity_x
            if y <= 0:
                y = 0
                velocity_y = -velocity_y
            if y >= 1.0:
                y = 1.0
                velocity_y = -velocity_y
            seq_x[i], seq_y[i] = x, y
        # Scale to the size.
        seq_x = (size * seq_x).astype(np.int32)
        seq_y = (size * seq_y).astype(np.int32)
        return seq_y, seq_x

    def random_mmnist(self):
        '''generate frames of moving mnist. '''
        data = np.zeros((20, 64, 64), dtype=np.float32)
        # 10 input + 10 groundtruth = 20
        for n in range(2):
            seq_y, seq_x = self.gen_random_sequence()
            idx = random.randint(0, self.mnist_train_im.shape[0] - 1)
            mnist_image = self.mnist_train_im[idx]
            for i in range(20):
                # put the 2 moving digits into "data"
                data[i, seq_y[i]:seq_y[i]+28, seq_x[i]:seq_x[i]+28] = np.maximum(
                  data[i, seq_y[i]:seq_y[i]+28, seq_x[i]:seq_x[i]+28],
                  mnist_image)
        return data

    def __getitem__(self, idx):
      if self.data_set == 'train':
        a = tf.expand_dims(self.random_mmnist()[..., None],0)
        for i in range(self.batch_size-1):
              a = tf.concat([a,tf.expand_dims(self.random_mmnist()[..., None],0)],0) 
        a = a/255
        input = tf.convert_to_tensor(a[:, 0:10], dtype='float')
        gt = tf.convert_to_tensor(a[:, 10:20], dtype='float')
      elif self.data_set == 'test':
        temp = self.mnist_test
        index = idx*16
        temp = temp[index:index+16]
        temp = temp/255
        input = tf.convert_to_tensor(temp[:, 0:10], dtype='float')
        gt = tf.convert_to_tensor(temp[:, 10:20], dtype='float')
      else:
        temp = self.mnist_valid
        index = idx*16
        temp = temp[index:index+16]
        temp = temp/255
        input = tf.convert_to_tensor(temp[:, 0:10], dtype='float')
        gt = tf.convert_to_tensor(temp[:, 10:20], dtype='float')

      return input, gt
      


In [16]:
train_set = MMNISTDataset('train')
test_set = MMNISTDataset('test')
valid_set = MMNISTDataset('valid')

(20, 10000, 64, 64)


In [17]:
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML

def visualize(video):
    video = video.reshape(10,64,64)

    fig = plt.figure()
    im = plt.imshow(video[0,:,:])

    plt.close() # this is required to not display the generated image

    def init():
        im.set_data(video[0,:,:])

    def animate(i):
        im.set_data(video[i,:,:])
        return im

    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=video.shape[0],
                                  interval=80)
    return HTML(anim.to_html5_video())

In [36]:
print(type(train_set[0][0][0]))
print(train_set[0][0][0].shape)

<class 'tensorflow.python.framework.ops.EagerTensor'>
(10, 64, 64, 1)


In [19]:
print(type(test_set[0][0]))
print(test_set[0][0].shape)

<class 'tensorflow.python.framework.ops.EagerTensor'>
(16, 10, 64, 64, 1)


In [22]:
visualize(test_set[0][0][0])

In [23]:
class Encoder(tf.keras.layers.Layer):

  def __init__(self):
    super(Encoder, self).__init__()
    self.enc = [
        Enc_ConvB(64,(1,1)), # as the same H & W
        Enc_ConvB(64,(2,2)), # as output with H/2 & W/2
        Enc_ConvB(64,(1,1)), # as the same with above one
        Enc_ConvB(64,(2,2)) # as output with H/4 & W/4
    ]

  def call(self, x):
    # print('The x_encoder_in shape: ', x.shape)
    x = self.enc[0](x)
    self.skip_connection = x
    x = self.enc[1](x)
    x = self.enc[2](x)
    x = self.enc[3](x)
    # print('The x_encoder_out shape: ', x.shape)
    return x, self.skip_connection

In [24]:
class Enc_ConvB(tf.keras.layers.Layer):
  def __init__(self, out_channels_size, stride_size):
      super(Enc_ConvB, self).__init__()
      self.conv = tf.keras.layers.Conv2D(
          filters=out_channels_size, 
          strides=stride_size, 
          kernel_size=(3,3), 
          padding='same')
      self.norm = tfa.layers.GroupNormalization(groups=2, 
                                                epsilon=1e-5)
      self.act = tf.keras.layers.LeakyReLU(alpha=0.2)
    
  def call(self, x):
    x = self.conv(x)
    x = self.norm(x)
    x = self.act(x)
    return x


In [25]:
class Translator(tf.keras.layers.Layer):
  def __init__(self):
    super(Translator, self).__init__()
    self.enc = [
        Inception(256),
        Inception(256),
        Inception(256),
        Inception(256),
        Inception(256),
        Inception(256),
        Inception(256),
        Inception(256)
    ]
    self.dec = [
        Inception(512),
        Inception(512),
        Inception(512),
        Inception(512),
        Inception(512),
        Inception(512),
        Inception(512),
        Inception(640)
    ]
  
  def call(self, x):
    # print('The x_translator_in shape: ', x.shape)
    x = tf.reshape(x, shape=(16, 10, 16, 16, 64))
    x = tf.transpose(x, [0,2,3,1,4])
    # print('The x_translator_after_transpose_1 shape: ', x.shape)
    x = tf.reshape(x, shape=(16, 16, 16, 10*64))
    IncepEnc_1 = self.enc[0](x)
    # print("The x_IncepEnc_1 shape: ", IncepEnc_1.shape)
    IncepEnc_2 = self.enc[1](IncepEnc_1)
    # print("The x_IncepEnc_2 shape: ", IncepEnc_2.shape)
    IncepEnc_3 = self.enc[2](IncepEnc_2)
    IncepEnc_4 = self.enc[3](IncepEnc_3)
    IncepEnc_5 = self.enc[4](IncepEnc_4)
    IncepEnc_6 = self.enc[5](IncepEnc_5)
    IncepEnc_7 = self.enc[6](IncepEnc_6)
    IncepEnc_8 = self.enc[7](IncepEnc_7)

    IncepDec_1 = self.dec[0](IncepEnc_8,)
    temp = tf.concat((IncepDec_1, IncepEnc_7), axis=3)
    IncepDec_2 = self.dec[1](temp)
    temp = tf.concat((IncepDec_2, IncepEnc_6), axis=3)
    IncepDec_3 = self.dec[2](temp)
    temp = tf.concat((IncepDec_3, IncepEnc_5), axis=3)
    IncepDec_4 = self.dec[3](temp)
    temp = tf.concat((IncepDec_4, IncepEnc_4), axis=3)
    IncepDec_5 = self.dec[4](temp)
    temp = tf.concat((IncepDec_5, IncepEnc_3), axis=3)
    IncepDec_6 = self.dec[5](temp)
    temp = tf.concat((IncepDec_6, IncepEnc_2), axis=3)
    IncepDec_7 = self.dec[6](temp)
    temp = tf.concat((IncepDec_7, IncepEnc_1), axis=3)
    IncepDec_8 = self.dec[7](temp)

    x = IncepDec_8.reshape(16,16,16,10,64)
    # print('The x_translator_after_all_inceptions shape: ', x.shape)
    x = tf.transpose(x, [0,3,1,2,4])
    # print('The x_translator_out shape: ', x.shape)
    

    return x




In [26]:
class Inception(tf.keras.layers.Layer):
  def __init__(self, C_out):
    super(Inception, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(
        filters=128, 
        strides=1, 
        kernel_size=(1,1)) # as the same H & W
    self.layers = [
        GroupConv(C_out,(3,3)),
        GroupConv(C_out,(5,5)),
        GroupConv(C_out,(7,7)),
        GroupConv(C_out,(11,11))
    ]

  def call(self, x):
    x = self.conv1(x)
    y1 = self.layers[0](x)
    y2 = self.layers[1](x)
    y3 = self.layers[2](x)
    y4 = self.layers[3](x)
    return (y1+y2+y3+y4)


In [27]:
class GroupConv(tf.keras.layers.Layer):
  def __init__(self, C_out, k):
    super(GroupConv, self).__init__()
    self.conv = tf.keras.layers.Conv2D(filters=C_out, strides=(1,1), kernel_size=k, padding='same', groups=8)
    self.norm = tfa.layers.GroupNormalization(groups=8, epsilon=1e-5)
    self.act = tf.keras.layers.LeakyReLU(alpha=0.2)
  
  def call(self, x):
    x = self.conv(x)
    x = self.norm(x)
    x = self.act(x)
    return x

In [28]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self):
    super(Decoder, self).__init__()
    self.dec = [
        Dec_ConvB(64, (2,2), True), # double x & y
        Dec_ConvB(64, (1,1), False), # same x & y
        Dec_ConvB(64, (2,2), True), # double x & y
        Dec_ConvB(64, (1,1), False) # same x & y
    ]
    self.readout = tf.keras.layers.Conv2D(1, kernel_size = (1,1))
  
  def call(self, x, skip_connection):
    # print('The x_decoder_in shape: ', x.shape)
    x = tf.reshape(x, shape=(16*10, 16, 16, 64))
    x = self.dec[0](x)
    x = self.dec[1](x)
    x = self.dec[2](x)
    # print('The x_decoder_after dec_1_2_3 shape: ', x.shape)
    # print('The skip_connection shape: ', skip_connection.shape)
    x = tf.concat((x, skip_connection), axis=3)
    x = self.dec[3](x)
    x = self.readout(x)
    x = tf.reshape(x, shape=(16, 10, 64, 64, 1))
    # print('The x_decoder_out shape: ', x.shape)
    return x

In [29]:
class Dec_ConvB(tf.keras.layers.Layer):
  def __init__(self, C_out, s, transpose):
    super(Dec_ConvB, self).__init__()
    self.transpose = transpose
    if transpose == True:
      self.conv = tf.keras.layers.Conv2DTranspose(filters=C_out, strides=s, kernel_size=(3,3), padding='same', output_padding = (1,1))
    else:
      self.conv = tf.keras.layers.Conv2D(filters=C_out, strides=s, kernel_size=(3,3), padding='same')
    self.norm = tfa.layers.GroupNormalization(groups=2, epsilon=1e-5)
    self.act = tf.keras.layers.LeakyReLU(alpha=0.2)

  def call(self, x):
    x = self.conv(x)
    x = self.norm(x)
    x = self.act(x)
    return x


In [30]:
class Model(tf.keras.layers.Layer):
  def __init__(self):
    super(Model, self).__init__()
    self.enc = Encoder()
    self.hid = Translator()
    self.dec = Decoder()
  
  def call(self, x):
    x = tf.reshape(x, shape=(160, 64, 64, 1))
    # print('The x_raw shape: ', x.shape)
    x, skip_connection = self.enc(x)
    x = self.hid(x)
    Y = self.dec(x, skip_connection)
    return Y


In [31]:
model = tf.keras.Sequential(Model())

In [37]:
b = tf.ones((16, 10, 64, 64, 1))
model(b).shape

TensorShape([16, 10, 64, 64, 1])

In [33]:
model.compile(
    optimizer = "adam",
    loss = "mse",
    metrics = ['mse', 'mae']
)

In [38]:
model.evaluate(test_set)

 16/563 [..............................] - ETA: 31:09 - loss: 0.8433 - mse: 0.8433 - mae: 0.6453

KeyboardInterrupt: 

In [ ]:
# model.load_weights('/content/drive/MyDrive/Colab Notebooks/pa2_data/pretrained_main/pretrained_main')

In [ ]:
trained_model = model.fit(train_set, epochs=1, validation_data = valid_set)

In [ ]:
import joblib
joblib.dump(trained_model, 'trained_model.pkl', compress=9)